### New Code for 90%

In [6]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint


# path to the model weights files.
weights_path = '../vgg16_weights.h5'
#top_model_weights_path = 'fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150


In [7]:
train_data_dir = '../dataset/training_set/smo'
validation_data_dir = '../dataset/test_set/smo'
nb_train_samples = 2000
nb_validation_samples = 300
epochs = 50
batch_size = 16

In [8]:
# build the VGG16 network
# dimensions of our images.

#model = applications.VGG16(weights='imagenet', include_top=False)
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
print('Model loaded.')

Model loaded.


In [9]:
# build a classifier model to put on top of the convolutional model
#top_model.load_weights(weights_path)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))


In [10]:

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
#model.load_weights(weights_path)

NameError: name 'Model' is not defined

In [ ]:

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples,
    

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint


# path to the model weights files.
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'cyril.hdf5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'dataset/training_set/smo'
validation_data_dir = 'dataset/test_set/smo'

nb_train_samples = 2000
nb_validation_samples = 300
epochs = 100
batch_size = 32

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

In [ ]:

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
#top_model.load_weights(top_model_weights_path)


model.load_weights('cyril.h5',by_name=True)
#json_string = model.to_json()
#model = model_from_json(json_string)


In [ ]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [579]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')
model.summary()


Found 4299 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_80 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
___________________________________________________________

In [ ]:
# fine-tune the model
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
                            mode='max')
callbacks_list = [checkpoint]


# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples ,
    #steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples  ,
    #validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list)
    #verbose=2)


Epoch 1/100
2000/2000 [==============================] - 303s 152ms/step - loss: 0.4288 - acc: 0.8000 - val_loss: 0.4023 - val_acc: 0.8217

Epoch 00001: val_acc improved from -inf to 0.82167, saving model to every.best.hdf5
Epoch 2/100
2000/2000 [==============================] - 303s 152ms/step - loss: 0.3071 - acc: 0.8702 - val_loss: 0.3785 - val_acc: 0.8433

Epoch 00002: val_acc improved from 0.82167 to 0.84333, saving model to every.best.hdf5
Epoch 3/100
2000/2000 [==============================] - 301s 151ms/step - loss: 0.2362 - acc: 0.9046 - val_loss: 0.4265 - val_acc: 0.8400

Epoch 00003: val_acc did not improve from 0.84333
Epoch 4/100
2000/2000 [==============================] - 295s 147ms/step - loss: 0.1718 - acc: 0.9342 - val_loss: 0.4276 - val_acc: 0.8500

Epoch 00004: val_acc improved from 0.84333 to 0.85000, saving model to every.best.hdf5
Epoch 5/100
2000/2000 [==============================] - 305s 153ms/step - loss: 0.1132 - acc: 0.9589 - val_loss: 0.4743 - val_acc: 

In [ ]:
model.save_weights('cyril.hdf5
#model.save_weights('every.best.hdf5')